# **EDA gyakorlat**
Az alábbi notebook feltáró adatelemzés (*Exploratory Data Analysis - EDA*) főbb lépéseit, valamint az EDA riport fontosabb elemeit hivatott bemutatni.

Az EDA fontos része az adatelemzési munkának, ugyanis a folyamat során sok olyan összefüggést vagy jellemzőt ismerünk meg a vizsgált adathalmazzal kapcsolatban, melyek segítik a későbbi, mélyebb elemzést, mint például modellillesztést vagy hipotézis vizsgálatokat.

Fontos arra figyelni, hogy adatelemzési munka során nem csak egy adathalmazt kapunk, hanem azzal kapcsolatos kérdéseket, hipotéziseket is. A feladat elsősorban ezeknek a megválaszolása, így az analízis során ezeket fontos észben tartani, és azokra a jellemzőkre, összefüggésekre koncentrálni, melyek segítenek ebben.


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## **1. Szöveges bemutatása az adatszerkezetnek és a jellemzőknek**
EDA során első lépés, hogy szövegesen ismertetjük az adathalmazt, kontextust adunk a tartalmának, egyes jellemzőknek, értékeknek.

**Példa adatszerkezet:** Kaggle Car Features and MSRP adathalmaz (https://www.kaggle.com/datasets/CooperUnion/cardataset) 

Ez az adathalmaz különböző autókról tartalmaz leíró adatokat, mint például fogyasztás, műszaki paraméterek, gyártási év, és piaci jellemzők.
Minden bejegyzés egy specifikus autó fajtához tartozik.

Az autók összegyűjtött jellemzői:
* Márka
* Típus
* Gyártási év
* Üzemanyag típus
* Motor teljesítmény (lóerőben)
* Hengerek száma
* Váltó típusa
* Meghajtás (első, hátsó, összkerék)
* Ajtók száma
* Piaci kategória
* Jármű mérete
* Jármű kialakítása
* Országúti fogyasztás (MPG-ben)
* Városi fogyasztás (MPG-ben)
* Népszerűség (önkényes metrika, vásárlási, közösségi média, egyéb metrikák alapján)
* MSRP (*Manufacturer Suggested Retail Price*) gyártók által javasolt eladási ár (dollárban) 


Az adathalmaz készítője négy kérdést is feltett a vizsgálathoz:
1. Egyes jellemzők hatása az árra
2. A márka hogyan befolyásolja az árakat?
3. Melyik aoutók tekinthetők túlárazottnak?
4. Népszerűség kontra ár


## **2. Adatszerkezet beolvasása és megismerése**

In [2]:
# Kaggle-ről egy csv fájl formájában kerül letöltésre
df = pd.read_csv("data.csv")
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [3]:
# adathalmaz mérete
df.shape

(11914, 16)

In [4]:
# oszlopok
df.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

In [5]:
# oszlopok típusa (szöveg, objektum, szám)
df.dtypes

Make                  object
Model                 object
Year                   int64
Engine Fuel Type      object
Engine HP            float64
Engine Cylinders     float64
Transmission Type     object
Driven_Wheels         object
Number of Doors      float64
Market Category       object
Vehicle Size          object
Vehicle Style         object
highway MPG            int64
city mpg               int64
Popularity             int64
MSRP                   int64
dtype: object

Fontos megkülönböztetni a pandas által értelmezett típust az adott jellemző valódi típusától. 

Pl. érdekes, hogy az ajtók és a hengerek is valós számokként szerepelnek, miközben 4.5 hengeres autó nem létezik...

Hasonlóan most a pandas az összes szöveges mezőt objectnek kezelte

***Duplikátumok vizsgálata***

In [6]:
# mennyi duplikátum van az adathalmazban
df[df.duplicated()]

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
14,BMW,1 Series,2013,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,19,3916,31500
18,Audi,100,1992,regular unleaded,172.0,6.0,MANUAL,front wheel drive,4.0,Luxury,Midsize,Sedan,24,17,3105,2000
20,Audi,100,1992,regular unleaded,172.0,6.0,MANUAL,front wheel drive,4.0,Luxury,Midsize,Sedan,24,17,3105,2000
24,Audi,100,1993,regular unleaded,172.0,6.0,MANUAL,front wheel drive,4.0,Luxury,Midsize,Sedan,24,17,3105,2000
25,Audi,100,1993,regular unleaded,172.0,6.0,MANUAL,front wheel drive,4.0,Luxury,Midsize,Sedan,24,17,3105,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11481,Suzuki,X-90,1998,regular unleaded,95.0,4.0,MANUAL,four wheel drive,2.0,NaN,Compact,2dr SUV,26,22,481,2000
11603,Volvo,XC60,2017,regular unleaded,302.0,4.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Luxury,Performance",Midsize,4dr SUV,29,20,870,46350
11604,Volvo,XC60,2017,regular unleaded,240.0,4.0,AUTOMATIC,front wheel drive,4.0,"Crossover,Luxury",Midsize,4dr SUV,30,23,870,40950
11708,Suzuki,XL7,2008,regular unleaded,252.0,6.0,AUTOMATIC,all wheel drive,4.0,Crossover,Midsize,4dr SUV,22,15,481,29149


In [7]:
duplicate_rows_df = df[df.duplicated()]
print("Duplikált sorok száma: ", duplicate_rows_df.shape[0])

Duplikált sorok száma:  715


Fontos értelmezni, hogy van-e értelme a duplikátumoknak az adott adathalmaz esetén.

Jelen esetben nincs.

***Mezők megszámolása***

In [8]:
df.count()

Make                 11914
Model                11914
Year                 11914
Engine Fuel Type     11911
Engine HP            11845
Engine Cylinders     11884
Transmission Type    11914
Driven_Wheels        11914
Number of Doors      11908
Market Category       8172
Vehicle Size         11914
Vehicle Style        11914
highway MPG          11914
city mpg             11914
Popularity           11914
MSRP                 11914
dtype: int64

*Vagy fordítva: üres mezők megszámolása*

In [9]:
df.isnull().sum()

Make                    0
Model                   0
Year                    0
Engine Fuel Type        3
Engine HP              69
Engine Cylinders       30
Transmission Type       0
Driven_Wheels           0
Number of Doors         6
Market Category      3742
Vehicle Size            0
Vehicle Style           0
highway MPG             0
city mpg                0
Popularity              0
MSRP                    0
dtype: int64

In [10]:
# üres mezők aránya (százalékosan)
df.isnull().sum() / df.shape[0] * 100

Make                  0.000000
Model                 0.000000
Year                  0.000000
Engine Fuel Type      0.025180
Engine HP             0.579151
Engine Cylinders      0.251805
Transmission Type     0.000000
Driven_Wheels         0.000000
Number of Doors       0.050361
Market Category      31.408427
Vehicle Size          0.000000
Vehicle Style         0.000000
highway MPG           0.000000
city mpg              0.000000
Popularity            0.000000
MSRP                  0.000000
dtype: float64

Itt fontos észrevétel, hogy a *Market Category* jellemző közel harmada üres.

## **3. Jellemzők vizsgálata és ábrázolása**

### **3.1. Numerikus adatok áttekintő vizsgálata**

In [11]:
# numerikus jellemzők főbb statiszikái
df.describe()

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP
count,11914.000000,11845.00000,11884.000000,11908.000000,11914.000000,11914.000000,11914.000000,1.191400e+04
mean,2010.384338,249.38607,5.628829,3.436093,26.637485,19.733255,1554.911197,4.059474e+04
std,7.579740,109.19187,1.780559,0.881315,8.863001,8.987798,1441.855347,6.010910e+04
min,1990.000000,55.00000,0.000000,2.000000,12.000000,7.000000,2.000000,2.000000e+03
25%,2007.000000,170.00000,4.000000,2.000000,22.000000,16.000000,549.000000,2.100000e+04
50%,2015.000000,227.00000,6.000000,4.000000,26.000000,18.000000,1385.000000,2.999500e+04
75%,2016.000000,300.00000,6.000000,4.000000,30.000000,22.000000,2009.000000,4.223125e+04
max,2017.000000,1001.00000,16.000000,4.000000,354.000000,137.000000,5657.000000,2.065902e+06


*pandas.describe* függvény automatikusan csak a szám típusú jellemzőket írja ki

Ezeket is értelmezni kell, például hol van értelme ezeknek a jellemzőknek (évnél van?)

További észrevételek:  *popularity, engine hp, MSRP* esetében nagy a szórás az átlaghoz képest 

További statisztikai jellemzők

*itt már nincs automatikus szűrés a számokra, manuálisan kell elvégezni*

In [12]:
# módusz
df.select_dtypes(exclude="object").mode()

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP
0,2015,200.0,4.0,4.0,24,17,1385,2000


In [13]:
# ferdeség
df.select_dtypes(exclude="object").skew()

Year                -1.221981
Engine HP            1.291230
Engine Cylinders     0.964542
Number of Doors     -0.968671
highway MPG          7.573931
city mpg             7.106681
Popularity           1.653443
MSRP                11.771987
dtype: float64

In [14]:
# csúcsosság
df.select_dtypes(exclude="object").kurt()

Year                  0.299838
Engine HP             2.323884
Engine Cylinders      1.974316
Number of Doors      -1.009111
highway MPG         181.627364
city mpg             74.623487
Popularity            2.277090
MSRP                268.926276
dtype: float64

***Jellemzők eloszlásának vizsgálata (MSRP, HP, Cylinders)***

In [15]:
fig = px.box(df['MSRP'])
fig.show()

In [16]:
px.histogram(df, x="MSRP")

In [17]:
fig = px.box(df['Engine HP'])
fig.show()

In [18]:
fig = px.box(df['Engine Cylinders'])
fig.show()

*Összes jellemző boxplot egy ábrába*

In [19]:
# numerikus jellemzők nevének kigyűjtése
cols = df.select_dtypes(exclude="object").columns

# megfelelő subplot készítése
fig = make_subplots(rows=1, cols = len(cols))

# boxplotok kirajzolása megfelelő subplotba
for i,col in enumerate(cols):
    fig.add_trace(go.Box(y=df[col], name=col,boxmean=True), row=1, col=i+1)

fig

*Hisztogramok*

In [20]:
cols = df.select_dtypes(exclude="object").columns

fig = make_subplots(rows=1, cols = len(cols))

for i,col in enumerate(cols):
    fig.add_histogram(x=df[col],row=1, col=i+1, name=col)

fig

Ha valamelyiket ezek közül érdekesnek találjuk, érdemes megnézni külön is.

Pl. Engine HP

In [21]:
# Engine HP eloszlása milyen?
px.histogram(df,x="Engine HP")

*Normális eloszlásra hasonlít, azonban nagyobb értékek felé elnyúlik. Esetleg log-normal eloszlás?*

In [22]:
# Logaritmikus skálán ábrázolva
px.histogram(df,x="Engine HP", log_x=True)

*Egyéb érdekesség: évszám*

In [23]:
# az évszámot érdemes megnézni közelebbről
px.histogram(df, x="Year")

Ezt ismét fontos értelmezni. 2015 óta ennyivel többfajta autót gyártottak? Sokkal valószínűbb, hogy az adatgyűjtés volt nem kiegyensúlyozott az évszám tekintetében.

### **3.2. Kategorikus változók vizsgálata**

***Kategóriák száma***

In [24]:
# nézzük meg, hogy a kategorikus változóknál mennyi kategória van
df.select_dtypes(include="object").nunique()

Make                  48
Model                915
Engine Fuel Type      10
Transmission Type      5
Driven_Wheels          4
Market Category       71
Vehicle Size           3
Vehicle Style         16
dtype: int64

viszonylag sok márka, és főképp sok típus van

In [ ]:
# statisztikai mutatók közül a módusz értelmezett
df.select_dtypes(include="object").mode()

***Kategórák eloszlásának (kiegyensúlyozottságának) vizsgálata***

Pl. melyik gyártó hány autót adott el?

In [ ]:
# mennyire egyenletes a kategóriák eloszlása
df['Make'].value_counts()

In [ ]:
px.bar(df['Make'].value_counts())

In [ ]:
# Autók kialakítása
df["Vehicle Style"].value_counts()

In [ ]:

px.bar(df["Vehicle Style"].value_counts())

***Arányok vizsgálata kördiagramon***

Autók típusa

In [ ]:
data_pie = df['Transmission Type'].value_counts()


fig = go.Figure(data=[go.Pie(labels=data_pie.index, values=data_pie.tolist(), textinfo='label+percent',
                             insidetextorientation='radial'
                            )])

fig.update_traces(hole=.3, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Pie chart of transmission type")

fig.show()

*Észrevétel: sok automata váltó (amerikai piac), másrészt pedig van egy UNKNOWN kategória is...*

Autók mérete kör diagramon

In [ ]:

data_pie = df['Vehicle Size'].value_counts()


fig = go.Figure(data=[go.Pie(labels=data_pie.index, values=data_pie.tolist(), textinfo='label+percent',
                             insidetextorientation='radial'
                            )])

fig.update_traces(hole=.3, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Pie chart of driven wheels variable")

fig.show()

Hengerek számának eloszlása

In [ ]:
fig = px.histogram(df, x="Engine Cylinders", title='Hengerek száma',)
fig.show()


Melyik autó az, ami a legtöbb hengerrel rendelkezik?

In [ ]:
index = np.argmax(df['Engine Cylinders'])
df.loc[index,:]

## **4. Változók közötti összefüggések vizsgálata**

Jellemzők egyenkénti értelmezése mellett vizsgálni kell a közöttük lévő esetleges összefüggéseket. Ez nagyban segít a későbbi tervezésben.

Ha a korábbi lépések során felmerült valami, esetleg rendelkezünk háttértudással, azok jó kiindulópontok lehetnek első lépésnek az összefüggés-vizsgálatban.

Ha nincs ilyen intuíció/ötlet, scatter matrix is jó kiindulásnak

In [ ]:
# változók közötti összefüggésekre ránézni
px.scatter_matrix(df.select_dtypes(exclude="object"))

Párat lehet érdemes megnézni

In [ ]:
# pl. fogyasztások
px.scatter(df, x="highway MPG", y ="city mpg")

*Arra számítunk, hogy a városi és országúti fogyasztás között lesz összefüggés, az adatok úgy tűnik alátámasztják ezt.* 

In [ ]:
# fogyaszás kontra javasolt ár
px.scatter(df, x="highway MPG", y ="MSRP")

*Esetleg inverz kapcsolat?*

In [ ]:
# fogyasztás kontra teljesítmény
px.scatter(df, x="highway MPG", y ="Engine HP")

***Autók árának alakulása az évek során***

In [ ]:
fig = px.box(df, x="Year", y="MSRP")

fig.update_layout(title="Árak statiszikái egyes években",
                  xaxis_title="Év",
                  yaxis_title="MSRP")

fig.show()

*Elsőre is kitűnik egy ugrás 2000 után...*

In [ ]:
# átlag és szórás kigyűjtése az egyes évekre
index = df.groupby(['Year'])['MSRP'].mean().index.tolist()
mean_prices = df.groupby(['Year'])['MSRP'].mean().values.tolist()
std_prices =  df.groupby(['Year'])['MSRP'].std().values.tolist()

price_per_year = pd.DataFrame(np.column_stack((mean_prices, std_prices)), columns=['Mean', 'Std'], index=index)

fig = go.Figure()

fig.add_trace(go.Scatter(x=index, y=price_per_year.Mean,
                    mode='markers',
                    name='Átlagár'))

fig.add_trace(go.Scatter(x=index, y=price_per_year.Std,
                    mode='markers',
                    name='Ár szórása'))

fig.update_layout(xaxis_title="Év",
                  yaxis_title="MSRP")

fig.show()

*Az 1990-es évek és 2000 között az árak 2000 USD körül koncentrálódnak. A 2000-es évektől kezdve az árak sokkal nagyobb mértékben változnak. Rendkívül nagy az eltérés 2008-ban, abban az évben, amikor a legdrágább autót adták el a piacon. A nagyon nagy eltéréseket mutató években rendszeresen nagyon drága autókat adtak el, szemben az átlagos árú járművek nagy többségével.*

Érdemes megvizsgálni a két időszakot külön

In [ ]:
# 2000 előtti autók statisztikái (kiugróan drága autók kiszűrése)
df_before_2000_filtered = df.loc[((df['Year']<=2000) & (df['MSRP']< 10000))]

fig = px.box(df_before_2000_filtered, x="Year", y="MSRP")

fig.update_layout(xaxis_title="Év",
                  yaxis_title="MSRP")

fig.show()

In [ ]:
# 2000 utáni autók statisztikái (kiugróan drága autók kiszűrésével)
df_after_2000_filtered = df.loc[((df['Year']>2000) & (df['MSRP']< 500000))]


fig = px.box(df_after_2000_filtered, x="Year", y="MSRP")

fig.update_layout(xaxis_title="Év",
                  yaxis_title="MSRP")

fig.show()

***Hogyan alakult a 10 legnépszerűbb márka esetén az ár?***

*(Emlékezve a márkával kapcsolatos kérdésre)*

In [ ]:
# adathalmazban szereplő 10 leggyakoribb márka
popular_labels = df["Make"].value_counts().index[:10]

data_to_display = df[['Make','Year','MSRP']].loc[(df['Make'].isin(popular_labels)) & (df['Year'] > 2000)]


fig = px.box(data_to_display, x="Year", y="MSRP")

fig.update_layout(title="MSRP a 10 leggyakoribb márka szerint",
                  xaxis_title="Év",
                  yaxis_title="MSRP")

fig.show()

***Autó típusok üzemanyag szerint***

In [ ]:
px.histogram(df, x="Engine Fuel Type")

Üzemanyagok eloszlása az évek szerint

In [ ]:
# adatok szétválogatása 2015 előtt és utánira
eng = df.loc[~df['Year'].isin([2015,2016,2017]),'Engine Fuel Type'].value_counts()
eng2 = df.loc[df['Year'].isin([2015,2016,2017]),'Engine Fuel Type'].value_counts()

# arányosítások
eng = eng /eng.sum()
eng2 = eng2 / eng2.sum()

In [ ]:
# két barplot egy ábrán graph objects segítségével
fig = go.Figure()

fig.add_trace(go.Bar(
    x=eng.index,
    y=eng.values,
    name='2015 előtti autók',
    marker_color='indianred'
))

fig.add_trace(go.Bar(
    x=eng2.index,
    y=eng2.values,
    name='2015 utáni autók',
    marker_color='purple'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title_text='Üzemanyag típust használó autók aránya')

fig.show()

***5K $-nál olcsóbb autók alakulása és átlag ára***

In [ ]:
# $5000 alatti autók száma és átlag ára évek szerint
below_5k_per_year_count = df.loc[df['MSRP']<5000].groupby(['Year'])['MSRP'].count()
below_5k_per_year_mean = df.loc[df['MSRP']<5000].groupby(['Year'])['MSRP'].mean()

In [ ]:
# Subplotok létrehozása
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=below_5k_per_year_mean.index, 
               y=below_5k_per_year_mean, 
               name="Átlagos MSRP",
               marker_color='indianred'),
    row=1,
    col=1
)

fig.add_trace(go.Bar(
    x=below_5k_per_year_count.index,
    y=below_5k_per_year_count,
    name='Autók száma',
    marker_color='lightsalmon'
),
             row=1,
             col=2)

fig.update_xaxes(title_text="Év")

fig.update_yaxes(title_text="Autók száma", row=1, col=2 )
fig.update_yaxes(title_text="Átlagos MSRP", row=1, col=1 )

fig.update_layout(barmode='group', 
                  xaxis_tickangle=-45,
                  title_text='5000 USD alatti autók jellemzői',
                  showlegend=False)

fig.show()

***Korrelációs mátrix***

Egyes (numerikus) jellemzők közötti **lineáris** összefüggést vizsgálja

In [ ]:
df.select_dtypes(exclude="object").corr()

Ábrán szemléletesebb

In [ ]:
# hol vannak lineáris összefüggések?
px.imshow(df.select_dtypes(exclude="object").corr(), text_auto=True)


Erős *lineáris* korreláció:
- **Engine cylinders & Engine HP**
- **highway mpg & City mpg**
- **Engine HP és MSRP**

Minél több henger van, annál erősebb az autó.

Erős negatív korreláció:
- **Engine cylinders & highway mpg**

Erős negatív korrelációt mutatnak az országúti és városi MPG értékekkel a hengerek száma, mivel az alacsonyabb MPG értékek magasabb üzemanyag-fogyasztást jelentenek.

Fontos, hogy jelenleg csak a lineáris összefüggéseket vizsgáltuk. Az alacsony értékek nem zárják ki, hogy a jellemzők között egyéb összefüggés sincs (polinomiális, logaritmikus, stb.)